In [3]:
from supabase import create_client, Client
import os

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(supabase_url, supabase_key)

In [4]:
import pandas as pd

In [5]:
response = (
    supabase.table("Payments")
    .select("*")
    .csv()
    .execute()
)

In [6]:
response.data

'id,created_at,uuid,product_name,quantity,price,purchase_date,tags\n34,"2025-04-12 12:13:44.367295+00",47823327-2b0f-48c8-9513-614c3ab5d61a,Молоко,2,3,2023-10-01,"{еда,молочные}"\n35,"2025-04-12 12:13:44.367295+00",47823327-2b0f-48c8-9513-614c3ab5d61a,Хлеб,1,1,2023-10-01,"{еда,хлебобулочные}"\n36,"2025-04-12 12:13:44.367295+00",47823327-2b0f-48c8-9513-614c3ab5d61a,Яблоки,3,2,2023-10-02,"{фрукты,съедобное}"\n37,"2025-04-12 12:14:20.19878+00",47823327-2b0f-48c8-9513-614c3ab5d61a,Молоко,2,200,2025-04-12,{food}'

In [7]:
from io import StringIO

In [8]:
df = pd.read_csv(StringIO(response.data), index_col=0, parse_dates=['purchase_date'])
df.drop(columns=['created_at'], inplace=True)
df

,uuid,product_name,quantity,price,purchase_date,tags
id,,,,,,
34,47823327-2b0f-48c8-9513-614c3ab5d61a,Молоко,2,3,2023-10-01,"{еда,молочные}"
35,47823327-2b0f-48c8-9513-614c3ab5d61a,Хлеб,1,1,2023-10-01,"{еда,хлебобулочные}"
36,47823327-2b0f-48c8-9513-614c3ab5d61a,Яблоки,3,2,2023-10-02,"{фрукты,съедобное}"
37,47823327-2b0f-48c8-9513-614c3ab5d61a,Молоко,2,200,2025-04-12,{food}


In [9]:
df.dtypes

uuid                     object
product_name             object
quantity                  int64
price                     int64
purchase_date    datetime64[ns]
tags                     object
dtype: object

In [10]:
test_uuid = "47823327-2b0f-48c8-9513-614c3ab5d61a"

In [11]:
tmp = df[df['uuid'] == test_uuid][['product_name', 'quantity', 'price', 'purchase_date']]
tmp = tmp.groupby(['product_name', 'purchase_date']).sum().reset_index().sort_values(by='purchase_date')

tmp

,product_name,purchase_date,quantity,price
0,Молоко,2023-10-01,2,3
2,Хлеб,2023-10-01,1,1
3,Яблоки,2023-10-02,3,2
1,Молоко,2025-04-12,2,200


In [12]:
tmp['purchase_date'].unique().tolist()

[Timestamp('2023-10-01 00:00:00'),
 Timestamp('2023-10-02 00:00:00'),
 Timestamp('2025-04-12 00:00:00')]

In [13]:
import pandas as pd
import json

# Предположим, что tmp уже существует
# tmp = df[df['uuid'] == test_uuid][['product_name', 'quantity', 'price', 'purchase_date']]
# tmp = tmp.groupby(['product_name', 'purchase_date']).sum()

# Сброс индекса, чтобы сделать product_name и purchase_date обычными колонками
tmp_reset = tmp.reset_index()

# Создание списка уникальных дат
labels = tmp_reset['purchase_date'].astype(str).unique().tolist()

# Подготовка данных для Quantity
quantity_datasets = []
for product_name, group in tmp_reset.groupby('product_name'):
    quantity_datasets.append({
        "label": product_name,
        "data": group.set_index('purchase_date')['quantity'].reindex(labels, fill_value=0).tolist()
    })

# Подготовка данных для Price
price_datasets = []
for product_name, group in tmp_reset.groupby('product_name'):
    price_datasets.append({
        "label": product_name,
        "data": group.set_index('purchase_date')['price'].reindex(labels, fill_value=0).tolist()
    })

# Формирование финального JSON
chart_data = {
    "quantity": {
        "labels": labels,
        "datasets": quantity_datasets
    },
    "price": {
        "labels": labels,
        "datasets": price_datasets
    }
}

# Вывод JSON в читаемом формате
print(json.dumps(chart_data, indent=4))

{
    "quantity": {
        "labels": [
            "2023-10-01",
            "2023-10-02",
            "2025-04-12"
        ],
        "datasets": [
            {
                "label": "\u041c\u043e\u043b\u043e\u043a\u043e",
                "data": [
                    2,
                    0,
                    2
                ]
            },
            {
                "label": "\u0425\u043b\u0435\u0431",
                "data": [
                    1,
                    0,
                    0
                ]
            },
            {
                "label": "\u042f\u0431\u043b\u043e\u043a\u0438",
                "data": [
                    0,
                    3,
                    0
                ]
            }
        ]
    },
    "price": {
        "labels": [
            "2023-10-01",
            "2023-10-02",
            "2025-04-12"
        ],
        "datasets": [
            {
                "label": "\u041c\u043e\u043b\u043e\u043a\u043e",
    

In [ ]:
user = pd.read_csv('data/user_1.csv', parse_dates=['real_transaction_dttm'])
user['real_transaction_dttm'] = user['real_transaction_dttm'].dt.date
user['loyalty_cashback_category_nm'] = user['loyalty_cashback_category_nm'].apply(lambda x: '{' + str(x) + '}')
user = user.drop(columns=['account_rk', 'transaction_type_cd'])
user.columns = ["price", "purchase_date", "tags"]
user['price'] = user['price'].astype(int)
user.sample(100).to_csv('data/user_1_capypay.csv', index=False)
user.head()

,price,purchase_date,tags
0,115,2023-07-08,{Рестораны}
1,1650,2023-07-07,{Фастфуд}
2,1550,2023-07-05,{Фастфуд}
3,60,2023-07-04,{Транспорт}
4,200,2023-07-04,{Отели}
